## Setup: Create a path at the root as ./Private/Data, and move the shared CSV dataset files into this path. The same directory has been gitignored to preserve data integrity. Once this has been completed, the cells can be run in order to create a single pandas dataframe with all the company data, which is then saved as a single master CSV file

## Please do NOT edit this notebook unless you want to modify the data retrival process. For approaches to use this data to predict valuations/other metrics, please create separate notebooks in this same master directory. This notebook is inteded for data retrieval and data cleaning alone

## Please also do not uncomment commented lines and then commit changes without re-commenting these lines. The lines have been commented in order to preserve data integrity, as this is a public repository.

## For more information about this project/access to the data, please contact LionBase at https://lionbase.nyc/

In [1]:
# Standard Imports
import pandas as pd
#import matplotlib.pyplot as plt
#import numpy as np
%matplotlib inline

path = './Private/Data/'
file_prefixes = ['Financial_', 'Market_', 'Profiles_']
file_extension = '.csv'
file_count = 6
#file_names = ['Financials.csv','MarketData.csv','Profiles.csv']

In [2]:
# Extracting these files into three frames:
financial_frame = pd.read_csv(path+file_prefixes[0]+'1'+file_extension)
market_frame = pd.read_csv(path+file_prefixes[1]+'1'+file_extension)
profile_frame = pd.read_csv(path+file_prefixes[2]+'1'+file_extension)
total_rows = len(financial_frame)

for i in range(1,6):
    file_no = i+1
    new_f_frame = pd.read_csv(path+file_prefixes[0]+str(file_no)+file_extension)
    total_rows += len(new_f_frame)
    new_m_frame = pd.read_csv(path+file_prefixes[1]+str(file_no)+file_extension)
    new_p_frame = pd.read_csv(path+file_prefixes[2]+str(file_no)+file_extension)
    financial_frame = financial_frame.append(new_f_frame)
    market_frame = market_frame.append(new_m_frame)
    profile_frame = profile_frame.append(new_p_frame)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [3]:
# Verifying that we have the same number of elements
len(market_frame) == len(profile_frame) == len(financial_frame)

True

In [4]:
master_frame = pd.concat([financial_frame,market_frame, profile_frame], axis=1)

In [5]:
def clean_ticker(old_ticker):
    old_ticker=str(old_ticker)
    if(old_ticker=='nan'):
        return "N/A"
    hyphen_index = old_ticker.find('-')
    if(hyphen_index==-1):
        return old_ticker
    new_ticker = old_ticker[0:hyphen_index]
    return new_ticker

In [6]:
master_frame.columns = master_frame.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
master_frame.drop(['name'],axis=1,inplace=True)
master_frame.drop(['cusip'],axis=1,inplace=True)
master_frame.drop(['tf.tkoactive'],axis=1,inplace=True)
master_frame.drop(['ds.instrument_type'],axis=1,inplace=True)
master_frame.drop(['tf.employees.footnote[cpa]'],axis=1,inplace=True)
master_frame.drop(['tf.employees[cpa]'],axis=1,inplace=True)
master_frame.drop(['tf.entity_name'],axis=1,inplace=True)

master_frame['quote_symbol'] = master_frame['quote_symbol'].apply(clean_ticker)
#master_frame.head()

In [7]:
master_frame.shape

(20740, 18)

### List of features avalible in the dataset

In [8]:
feature_list = list(master_frame.columns.values)
for feature in feature_list:
    print(feature)
print('\n')
print("Number of elements:{}".format(len(master_frame)))

mastercsv = path+'MasterDataset.csv'

current_sales
current_ebitda
current_ebit
current_net_income
current_total_assets
current_total_liabilities
current_market_cap_usd
current_price_close
current_pe_ratio
actual_eps
current_price_/_cash
current_price_/_sales
dividend_yield
country
exchange
primary_sic_code
quote_symbol
sedol


Number of elements:20740


In [9]:
#Save dataframe to CSV file
master_frame.to_csv(mastercsv, sep='\t', encoding='utf-8')

## This dataset is now ready to be used in a separate notebook, in order to analyse and predict trends of companies.